In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
 from fastcore.utils import *

In [ ]:
!pip install dtreeviz

In [ ]:
# from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG


In [ ]:
filepath = '/kaggle/input/playground-series-s4e6'

In [ ]:
df = pd.read_csv(f"{filepath}/train.csv")
df

In [ ]:
df.columns

In [ ]:
dep_var = 'Target'


Dependent variable is the 'the thing' we are predicting , here it is 'Target'.

In [ ]:
X = df.drop('Target', axis=1)
y = df['Target']

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
Here we are label encoding the target variable , as all the columns are numerical except the 'Target' column . 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

Seperating the training and testing sets ..

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

For this classification problem we are certainly be using the DecisionTree because they consume less time and all are numerical data so there will be easy in splitting up the nodes . 


In [ ]:
y_pred = model.predict(X_test)

Making the predictions 

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")



In [ ]:
y_pred_labels = label_encoder.inverse_transform(y_pred)
print("Predicted labels:", y_pred_labels)

The basic implementation gives an accuracy of : 74% 

In [ ]:
procs = [Categorify, FillMissing]
cont,cat = cont_cat_split(df,1,dep_var)

Categorify will encode the category variables (variables that are non numerical into some type of numeric value) and FillMissing will add some values in the empty fields of the dataset.

In [ ]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var)
len(to.train), len(to.valid)

In [ ]:
xs,y = to.train.xs , to.train.y 

In [ ]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(to.train.xs, to.train.y)


In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

We are checking what are the most important features that effect the output using the feature_importance that is already available in the DecisionTreeRegressor

In [ ]:
fi = rf_feat_importance(m, xs)
fi[:10]

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:30]);

Only 2 columns are really very important in determining the output . 


In [ ]:
to_keep = fi[fi.imp>0.035].cols
len(to_keep)

In [ ]:
xs_imp = xs[to_keep]

Dropping all the columns , except the important ones.

In [ ]:
xs_imp

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(xs_imp, y_encoded, test_size=0.2, random_state=42)

Re splitting them but now using the x_imp columns .

In [ ]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

The accuracy remains nearly same.

In [ ]:
test_df = pd.read_csv(f"{filepath}/test.csv")
test_df

In [ ]:
predictions = model.predict(test_df)
predictions

In [ ]:
y_pred_labels = label_encoder.inverse_transform(predictions)
y_pred_labels

In [ ]:
submission = pd.DataFrame({
    'id': test_df['id'],  # Assuming 'id' column exists in test.csv
    'Target': y_pred_labels  # Replace 'Target' with the actual target column name
})

In [ ]:
# submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission.Target.replace({2:"Graduate",0:"Dropout",1:'Enrolled'},inplace=True)

In [115]:
submission

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Enrolled
3,76521,Graduate
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


In [ ]:
submission.to_csv('submission.csv', index=False)